In [6]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et
from config import region

In [7]:
xml_list = glob(region + '/data/data_images/*.xml')

In [8]:
len(xml_list)

5012

In [9]:
xml_list = list(map(lambda x: x.replace('\\','/'), xml_list))

In [10]:
len(xml_list)

5012

In [11]:
tree = et.parse(xml_list[0])
root = tree.getroot()
print(root)
print(root.tag)

<Element 'annotation' at 0x108940040>
annotation


In [12]:
image_name = root.find('filename').text
print(image_name)

007826.jpg


In [13]:
width = root.find('size').find('width').text
height = root.find('size').find('height').text
print(width + ' x ' + height)

500 x 375


In [14]:
objs = root.findall('object')
objs

[<Element 'object' at 0x108940630>,
 <Element 'object' at 0x108940950>,
 <Element 'object' at 0x108940c70>,
 <Element 'object' at 0x108940f90>,
 <Element 'object' at 0x108a7c310>,
 <Element 'object' at 0x108a7c630>,
 <Element 'object' at 0x108a7c950>]

In [15]:
parser=[]
for obj in objs:
    name = obj.find('name').text
    bndbox = obj.find('bndbox')
    xmin = bndbox.find('xmin').text
    xmax = bndbox.find('xmax').text
    ymin = bndbox.find('ymin').text
    ymax = bndbox.find('ymax').text
    parser.append([image_name, width, height, name, xmin, ymin, xmax, ymax])
print(parser)

[['007826.jpg', '500', '375', 'diningtable', '80', '217', '320', '273'], ['007826.jpg', '500', '375', 'chair', '197', '193', '257', '326'], ['007826.jpg', '500', '375', 'chair', '139', '184', '185', '231'], ['007826.jpg', '500', '375', 'chair', '258', '180', '312', '314'], ['007826.jpg', '500', '375', 'chair', '10', '195', '93', '358'], ['007826.jpg', '500', '375', 'chair', '82', '252', '243', '372'], ['007826.jpg', '500', '375', 'chair', '43', '319', '144', '375']]


In [16]:
parse_all = reduce(lambda x, y: x+y, parser)
print(parse_all)

['007826.jpg', '500', '375', 'diningtable', '80', '217', '320', '273', '007826.jpg', '500', '375', 'chair', '197', '193', '257', '326', '007826.jpg', '500', '375', 'chair', '139', '184', '185', '231', '007826.jpg', '500', '375', 'chair', '258', '180', '312', '314', '007826.jpg', '500', '375', 'chair', '10', '195', '93', '358', '007826.jpg', '500', '375', 'chair', '82', '252', '243', '372', '007826.jpg', '500', '375', 'chair', '43', '319', '144', '375']


In [17]:
df = pd.DataFrame(parser, columns=['filename', 'width', 'height', 'name', 'xmin', 'ymin', 'xmax', 'ymax'])
df.head()

,filename,width,height,name,xmin,ymin,xmax,ymax
0,007826.jpg,500,375,diningtable,80,217,320,273
1,007826.jpg,500,375,chair,197,193,257,326
2,007826.jpg,500,375,chair,139,184,185,231
3,007826.jpg,500,375,chair,258,180,312,314
4,007826.jpg,500,375,chair,10,195,93,358


In [18]:
df.shape

(7, 8)

In [19]:
parser1=[]
for img_details in xml_list:
    tree1 = et.parse(img_details)
    root1 = tree1.getroot()
    objs1 = root1.findall('object')
    image_name1 = root1.find('filename').text
    width1 = root1.find('size').find('width').text
    height1 = root1.find('size').find('height').text
    for obj1 in objs1:
        name1 = obj1.find('name').text
        bndbox1 = obj1.find('bndbox')
        xmin1 = bndbox1.find('xmin').text
        xmax1 = bndbox1.find('xmax').text
        ymin1 = bndbox1.find('ymin').text
        ymax1 = bndbox1.find('ymax').text
        parser1.append([image_name1, width1, height1, name1, xmin1, ymin1, xmax1, ymax1])
df = pd.DataFrame(parser1, columns=['filename', 'width', 'height', 'name', 'xmin', 'ymin', 'xmax', 'ymax'])

In [20]:
df.head(15)

,filename,width,height,name,xmin,ymin,xmax,ymax
0,007826.jpg,500,375,diningtable,80,217,320,273
1,007826.jpg,500,375,chair,197,193,257,326
2,007826.jpg,500,375,chair,139,184,185,231
3,007826.jpg,500,375,chair,258,180,312,314
4,007826.jpg,500,375,chair,10,195,93,358
5,007826.jpg,500,375,chair,82,252,243,372
6,007826.jpg,500,375,chair,43,319,144,375
7,002786.jpg,500,332,horse,80,97,348,272
8,002786.jpg,500,332,person,201,52,258,202
9,006286.jpg,500,375,person,80,88,405,375


In [21]:
df.shape

(15663, 8)

In [22]:
len(xml_list)

5012

In [23]:
images = df['filename'].unique()
len(images)

5012

In [24]:
img_df = pd.DataFrame(images, columns=['filename'])
img_df

,filename
0,007826.jpg
1,002786.jpg
2,006286.jpg
3,002962.jpg
4,008297.jpg
...,...
5007,005782.jpg
5008,000822.jpg
5009,004488.jpg
5010,005796.jpg


In [25]:
img_train = tuple(img_df.sample(frac=0.8)['filename'])
len(img_train)

4010

In [26]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename'])
len(img_test)

1002

In [27]:
train_df = df.query('filename in @img_train')
test_df = df.query('filename in @img_test')

In [28]:
train_df.shape

(12594, 8)

In [29]:
test_df.shape

(3069, 8)

In [35]:
#Label encoding
def label_encoding(x):
    labels = {'person' : 0, 'car':1, 'chair': 2, 'bottle': 3, 'pottedplant':4, 'bird': 5, 'dog': 6,     'sofa':7, 'bicycle':8, 'horse':9, 'boat': 10, 'motorbike': 11, 'cat':12, 'tvmonitor':13, 'cow':14, 'sheep': 15, 'aeroplane': 16, 'train': 17, 'diningtable': 18, 'bus':19}
    return labels [x]

In [36]:
train_df.loc[:, 'id'] = train_df['name'].apply(label_encoding)

In [37]:
test_df.loc[:, 'id'] = test_df['name'].apply(label_encoding)

In [38]:
train_df.head(10)

,filename,width,height,name,xmin,ymin,xmax,ymax,id
0,007826.jpg,500,375,diningtable,80,217,320,273,18
1,007826.jpg,500,375,chair,197,193,257,326,2
2,007826.jpg,500,375,chair,139,184,185,231,2
3,007826.jpg,500,375,chair,258,180,312,314,2
4,007826.jpg,500,375,chair,10,195,93,358,2
5,007826.jpg,500,375,chair,82,252,243,372,2
6,007826.jpg,500,375,chair,43,319,144,375,2
7,002786.jpg,500,332,horse,80,97,348,272,9
8,002786.jpg,500,332,person,201,52,258,202,0
9,006286.jpg,500,375,person,80,88,405,375,0


In [39]:
test_df.head(10)

,filename,width,height,name,xmin,ymin,xmax,ymax,id
45,006523.jpg,500,375,aeroplane,189,186,312,230,16
69,000620.jpg,500,344,car,114,182,303,309,1
70,000620.jpg,500,344,car,51,267,71,280,1
71,000620.jpg,500,344,car,19,272,36,284,1
93,005016.jpg,500,333,aeroplane,10,121,196,183,16
94,005016.jpg,500,333,aeroplane,119,53,489,190,16
95,003301.jpg,500,333,person,97,41,271,333,0
96,003301.jpg,500,333,dog,104,196,236,294,6
97,003301.jpg,500,333,chair,96,190,291,333,2
98,003301.jpg,500,333,pottedplant,4,110,109,309,4
